# Multi-agent collaboration for student advising

In [ ]:
#!python3 -m pip install --force-reinstall --no-cache -q -r requirements.txt

In [ ]:
!pip install termcolor

# Agent Configuration

In [ ]:
agent_instruction = """
You are a supervisor agent responsible for coordinating between a text2sql agent and a course advisor agent. Your role is to:

1. Analyze user queries and determine which agent(s) should handle the request
2. Route requests to the appropriate agent(s)
3. Combine responses when needed
4. Ensure smooth interaction between agents when a task requires both agents

Guidelines for request handling:
1. For text2sql related queries (involving student information, course schedule, student schedule):
   - Route to the text2sql agent

2. For course and program related queries:
   - Route to the course advisor agent

3. For complex queries requiring both systems:
   - Break down the request into sub-tasks
   - Route each sub-task to the appropriate agent
   - Combine the responses in a meaningful way
   - Example: "What courses are offerred this semester that's relevant to student's major"

Response formatting:

1. Clearly indicate which agent provided which part of the response
2. Maintain context between related pieces of information
3. Present combined information in a logical and easy-to-understand format

Error handling:

1. If an agent cannot process a request, relay the error and suggest alternatives
2. If unsure about which agent should handle a request, ask the user for clarification
3. Ensure that partial failures don't prevent the delivery of available information

When interacting with users:
1. Maintain a helpful and professional tone
2. Clearly communicate which system is being queried
3. Ask for clarification when needed to route requests properly

Remember: Your primary role is to coordinate and ensure effective communication between the specialized agents while providing a seamless experience for the user.

"""

In [ ]:
# to retrieve text2sql_agent_alias_arn, course_advisor_agent_alias_arn
%store -r text2sql_agent_alias_arn
%store -r course_advisor_agent_alias_arn

In [ ]:
agent_name = 'course-recommendation-multi-agent'
agent_foundation_model = "anthropic.claude-3-5-sonnet-20241022-v2:0"
agent_description = "Multi-agent collaboration for course recommendation"
sub_agents_list = [
    {
        'sub_agent_alias_arn': text2sql_agent_alias_arn,
        'sub_agent_instruction': """Use this agent to handle retrieving information through sql queries and letting other agents handle their specific domains.""",
        'sub_agent_association_name': 'Text2sqlAssistant',
        'relay_conversation_history': 'DISABLED'
    },
    {
        'sub_agent_alias_arn': course_advisor_agent_alias_arn,
        'sub_agent_instruction': """Use this agent to handle course and program requirement related questions while letting other agents handle their specific domains.""",
        'sub_agent_association_name': 'CourseAssistant',
        'relay_conversation_history': 'DISABLED'
    }
]

# Create Agent

In [ ]:
from bedrock_agent_helper import AgentsForAmazonBedrock

agent = AgentsForAmazonBedrock()
supervisor_agent_id, supervisor_agent_alias_id, supervisor_agent_alias_arn = agent.create_agent(
    'testing-api-attemp', 
    agent_description,
    agent_instruction,
    model_ids=[agent_foundation_model],
    agent_collaboration='SUPERVISOR_ROUTER'
)

In [ ]:
supervisor_agent_alias_id, supervisor_agent_alias_arn = agent.associate_sub_agents(
    supervisor_agent_id, sub_agents_list
)
supervisor_agent_alias_id, supervisor_agent_alias_arn